# Importing Libraries

In [13]:
from sklearn import tree
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np
import re

# Modifying Input Data

In [2]:
trainingData = pd.read_csv("../Data/train.csv")
trainingData["Cabin"] = trainingData["Cabin"].apply(str)
trainingData["Embarked"] = trainingData["Embarked"].apply(str)

AgeBins = pd.IntervalIndex.from_tuples([(0,15),(15,30),(30,45),(45,60),(60,75),(75,90),(90,105),(105,120)])
TicketBins = pd.IntervalIndex.from_tuples([(0,50000), (50000,100000), (100000,150000),(150000,200000),(200000, 250000), (250000, 300000), (300000, 350000)])
CostBins = pd.IntervalIndex.from_tuples([(-1, 50),(50,100), (100,150), (150, 200), (200, 250), (250, 300), (300,350), (350,400), (400, 450), (450, 500), (500, 550)])

trainingData["AgeGroup"] = pd.cut(trainingData['Age'], bins=AgeBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
trainingData['TicketGroup'] = pd.cut(pd.to_numeric(trainingData['Ticket'], errors="coerce"), bins=TicketBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))
trainingData['CostGroup'] = pd.cut(trainingData['Fare'], bins=CostBins).cat.add_categories(pd.Interval(-2,-1)).fillna(pd.Interval(-2,-1))

def nameClass(row):
    if (re.search("Mr\.",row["Name"])):
        return("Mr.")
    elif (re.search("Mrs\.",row["Name"])):
        return("Mrs.")
    elif (re.search("Miss\.",row["Name"])):
        return("Miss.")
    else:
        return("No Title")
        
trainingData["Title"] = trainingData.apply(lambda row: nameClass(row), axis=1)
trainingData.drop(["Name", "Age", "Ticket", "Fare"], axis=1,inplace=True)

trainingData

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Cabin,Embarked,AgeGroup,TicketGroup,CostGroup,Title
0,1,0,3,male,1,0,nan,S,"(15, 30]","(-2, -1]","(-1, 50]",Mr.
1,2,1,1,female,1,0,C85,C,"(30, 45]","(-2, -1]","(50, 100]",Mrs.
2,3,1,3,female,0,0,nan,S,"(15, 30]","(-2, -1]","(-1, 50]",Miss.
3,4,1,1,female,1,0,C123,S,"(30, 45]","(100000, 150000]","(50, 100]",Mrs.
4,5,0,3,male,0,0,nan,S,"(30, 45]","(-2, -1]","(-1, 50]",Mr.
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,0,0,nan,S,"(15, 30]","(200000, 250000]","(-1, 50]",No Title
887,888,1,1,female,0,0,B42,S,"(15, 30]","(100000, 150000]","(-1, 50]",Miss.
888,889,0,3,female,1,2,nan,S,"(-2, -1]","(-2, -1]","(-1, 50]",Miss.
889,890,1,1,male,0,0,C148,C,"(15, 30]","(100000, 150000]","(-1, 50]",Mr.


# CV-Sets

In [10]:
cvTrain1 = trainingData[0:(4*trainingData["Survived"].count()//5)]
cvTest1 = trainingData[(4*trainingData["Survived"].count()//5):]

cvTrain2 = trainingData[0:(3*trainingData["Survived"].count()//5)].append(trainingData[(4*trainingData["Survived"].count()//5):])
cvTest2 = trainingData[(3*trainingData["Survived"].count()//5):(4*trainingData["Survived"].count()//5)]

cvTrain3 = trainingData[0:(2*trainingData["Survived"].count()//5)].append(trainingData[(3*trainingData["Survived"].count()//5):])
cvTest3 = trainingData[(2*trainingData["Survived"].count()//5):(3*trainingData["Survived"].count()//5)]

cvTrain4 = trainingData[0:trainingData["Survived"].count()//5].append(trainingData[(2*trainingData["Survived"].count()//5):])
cvTest4 = trainingData[trainingData["Survived"].count()//5:(2*trainingData["Survived"].count()//5)]

cvTrain5 = trainingData[trainingData["Survived"].count()//5:]
cvTest5 = trainingData[0:trainingData["Survived"].count()//5]

features = ["Pclass", "Sex", "SibSp", "Parch", "Cabin", "Embarked", "AgeGroup", "TicketGroup", "CostGroup", "Title"]

# Accuracy Function

In [11]:
def accuracy(data,model,result):
    predictions = model.predict(data)
    total = 0
    for pred in range(len(predictions)):
        if prediction[pred] == result[pred]:
            total+=1
    return total/len(data)

In [21]:
encoder = OrdinalEncoder()
encoder.fit(cvTrain1[features])
encodedData = encoder.transform(cvTrain1[features])

dTree = tree.DecisionTreeClassifier()
dTree.fit(encodedData, cvTrain1["Survived"])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')